In [11]:
# Importing necessary libraries for web scraping, browser automation, and data manipulation

from selenium import webdriver

from selenium.webdriver.chrome.service import Service

from selenium.webdriver.support.ui import WebDriverWait

from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup

import codecs

import re

from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.support.select import Select
import pandas as pd
import requests

# Defining the path for the ChromeDriver executable and initializing the Chrome WebDriver
DRIVER_PATH = "C:/Windows/chromedriver.exe"
service = Service(DRIVER_PATH)
global driver
driver = webdriver.Chrome(service=service)

# Navigating to the SEBI website
driver.get('https://www.sebi.gov.in/sebiweb/other/OtherAction.do?doPmr=yes')

In [12]:
# Importing additional libraries for working with dropdown menus in Selenium
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

# Extracting all dropdown values for the 'pmrId' field
dropdown= Select(driver.find_element(By.NAME, 'pmrId'))
dropdown_list=[option.get_attribute('value') for option in dropdown.options]


In [13]:
# Initialize an empty DataFrame to store the extracted data
try:
    df1=pd.DataFrame()
    # Iterating over each value in the dropdown list (skipping the first, typically a placeholder)
    for i in range(1,len(dropdown_list)):

        # Selecting a dropdown value for 'pmrId'
        select_element = Select(driver.find_element(By.NAME, 'pmrId'))
        select_element.select_by_value(dropdown_list[i])

        # Selecting the year 2024 in the 'year' dropdown
        select_element = Select(driver.find_element(By.NAME, 'year'))
        select_element.select_by_value("2024")

        # Selecting the month December in the 'month' dropdown
        select_element = Select(driver.find_element(By.NAME,'month'))
        select_element.select_by_value("12")

        # Waiting until the 'Submit' button is clickable and then clicking it
        wait = WebDriverWait(driver, 10)
        submit_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".go_area.go-area .go-search.go_search")))
        submit_button.click()

        # Getting the page source of the loaded page
        response=driver.page_source

        # Parsing the page source with BeautifulSoup
        soup= BeautifulSoup(response,'html.parser' )

        # Finding the table with class 'org-table-1' that contains the required data
        rows=soup.find(name='div',attrs={"class": "org-table-1"})

        # If the table exists, extract relevant data
        if rows!=None:
            # Extracting all rows from the table
            rows=rows.find_all('tr')

            # Extracting data for 'Clients' row (row index 2 in the table)
            clients_row = [float(td.text.strip()) for td in rows[2].find_all("td")[1:]]

            # Extracting data for 'AUM' row (row index 3 in the table)
            aum_row = [(td.text.strip()) for td in rows[3].find_all("td")]

            # Creating a dictionary with extracted data
            da = {
            "Company Name":dropdown_list[i],
            "Clients (Domestic PF/EPFO)":clients_row[0],
            "Clients (Domestic Corporates)":clients_row[1],
            "Clients (Domestic Non-Corporates)":clients_row[2],
            "Clients (Foreign Non-Residents)":clients_row[3],
            "Clients (Foreign FPI)":clients_row[4],
            "Clients (Foreign Others)":clients_row[5],
            "Clients (Total)":clients_row[6],

            "AUM (Domestic PF/EPFO)":aum_row[1],
            "AUM (Domestic Corporates)":aum_row[2],
            "AUM (Domestic Non-Corporates)":aum_row[3],
            "AUM (Foreign Non-Residents)":aum_row[4],
            "AUM (Foreign FPI)":aum_row[5],
            "AUM (Foreign Others)":aum_row[6],
            "AUM (Total)":aum_row[7]
            }

            # Converting the dictionary to a DataFrame
            df = pd.DataFrame(da,index=["row1"])

            # Concatenating the new DataFrame with the main DataFrame
            df1 =pd.concat([df1, df], axis=0)
        else:
             # If the table is not found, skip to the next iteration
            continue
        
# Catching and printing any exceptions that occur during the scraping process
except Exception as e:
        print("Error")

Error
